SOP0128 - Enable HDFS Encryption zones in Big Data Clusters.
============================================================

Description
-----------

Use this notebook to patch configmaps after Big Data Cluster upgrade to
enable HDFS encryption zones for Encryption At Rest.

Steps
-----

### Instantiate Kubernetes client

In [ ]:
# Instantiate the Python Kubernetes client into 'api' variable

import os
from IPython.display import Markdown

try:
    from kubernetes import client, config
    from kubernetes.stream import stream
except ImportError: 

    # Install the Kubernetes module
    import sys
    !{sys.executable} -m pip install kubernetes    
    
    try:
        from kubernetes import client, config
        from kubernetes.stream import stream
    except ImportError:
        display(Markdown(f'HINT: Use [SOP059 - Install Kubernetes Python module](../install/sop059-install-kubernetes-module.ipynb) to resolve this issue.'))
        raise

if "KUBERNETES_SERVICE_PORT" in os.environ and "KUBERNETES_SERVICE_HOST" in os.environ:
    config.load_incluster_config()
else:
    try:
        config.load_kube_config()
    except:
        display(Markdown(f'HINT: Use [TSG118 - Configure Kubernetes config](../repair/tsg118-configure-kube-config.ipynb) to resolve this issue.'))
        raise

api = client.CoreV1Api()

print('Kubernetes client instantiated')

### Get the namespace for the big data cluster

Get the namespace of the Big Data Cluster from the Kuberenetes API.

**NOTE:**

If there is more than one Big Data Cluster in the target Kubernetes
cluster, then either:

-   set \[0\] to the correct value for the big data cluster.
-   set the environment variable AZDATA\_NAMESPACE, before starting
    Azure Data Studio.

In [ ]:
# Place Kubernetes namespace name for BDC into 'namespace' variable

if "AZDATA_NAMESPACE" in os.environ:
    namespace = os.environ["AZDATA_NAMESPACE"]
else:
    try:
        namespace = api.list_namespace(label_selector='MSSQL_CLUSTER').items[0].metadata.name
    except IndexError:
        from IPython.display import Markdown
        display(Markdown(f'HINT: Use [TSG081 - Get namespaces (Kubernetes)](../monitor-k8s/tsg081-get-kubernetes-namespaces.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [TSG010 - Get configuration contexts](../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb) to resolve this issue.'))
        display(Markdown(f'HINT: Use [SOP011 - Set kubernetes configuration context](../common/sop011-set-kubernetes-context.ipynb) to resolve this issue.'))
        raise

print('The kubernetes namespace for your big data cluster is: ' + namespace)

### Common utilities

In [ ]:
import base64
import difflib
import json
import kubernetes.client

master_pod_name = 'master-0'
hdfs_name_node_pod_name = 'nmnode-0-0'
storage_configmap_name = 'mssql-hadoop-storage-0-configmap'

def run_command_on_pod(pod, container, command):
    return stream(api.connect_get_namespaced_pod_exec, pod, namespace, command=['/bin/sh', '-c', command], container=container, stderr=True, stdout=True)

def get_security_json():
    # api.connect_get_namespaced_pod_exec returns malformed json with single quotes if output is a valid json
    #
    return json.loads(base64.b64decode(run_command_on_pod(
        pod=master_pod_name, container='mssql-server', command='cat /var/run/configmaps/cluster/security.json | base64')).decode('utf-8'))

def is_ad_deployment():
    security_json = get_security_json()

    if 'environmentVariables' in security_json:
        pod_vars = security_json['environmentVariables']
        if 'SECURITY_MODE' in pod_vars and pod_vars['SECURITY_MODE'] == 'ActiveDirectory':
            print ('The cluster is deployed with Active Directory!')
            return True
    return False

def is_bdc_kms_provider_present():
    present_message = 'BDC key provider jar exists'
    absent_message = 'BDC key provider jar does not exist'
    jar_presence = run_command_on_pod(
        pod=hdfs_name_node_pod_name, container='hadoop', command=
            'ls -la /opt/hadoop/share/hadoop/common/keyprovider/bdc-hadoop-kms-provider-*.jar && ' \
            f'echo "{present_message}" || echo "{absent_message}"')
    return present_message in jar_presence

def get_host_fqdn(host):
    security_json = get_security_json()

    if 'environmentVariables' in security_json:
        pod_vars = security_json['environmentVariables']

        for domain_var in ['SUBDOMAIN', 'DOMAIN']:
            if domain_var in pod_vars and pod_vars[domain_var] != '':
                host = f'{host}.{pod_vars[domain_var]}'
    return host

def configmap_exist(configmap_name):
    try:
        configmap = api.read_namespaced_config_map(name=configmap_name, namespace=namespace)
    except kubernetes.client.rest.ApiException:
        return False
    return True

def read_configmap(configmap_name):
    return api.read_namespaced_config_map(name=configmap_name, namespace=namespace)

def replace_configmap(configmap_name, patched_configmap):
    api.replace_namespaced_config_map(name=configmap_name, namespace=namespace, body=json.loads(patched_configmap)) # works!

def can_configmap_patch_be_applied():
    if not is_ad_deployment():
        return False, 'Warning! Configmap patch can only enable HDFS encryption zone on BDC deployments with Active Directory! Current deployment is non-AD.'
    
    if not is_bdc_kms_provider_present():
        return False, 'Warning! Configmap patch can only enable HDFS encryption zone on BDC deployments with BDC key provider for KMS.'

    if not configmap_exist(storage_configmap_name):
        return False, f'Warning! Configmap patch cannot be applied because BDC storage configmap {storage_configmap_name} does not exist.'
    
    return True, 'Ok'

### Patching configmaps `mssql-hadoop-storage-0-configmap`, `mssql-hadoop-sparkhead-configmap`, `mssql-hadoop-spark-0-configmap` after BDC upgrade from earlier versions to enable HDFS encryption zones support in Big Data Cluster.

In [ ]:
import re
import xml.etree.ElementTree as ET

# Preserving existing comments in XML document
#
class CommentedTreeBuilder(ET.TreeBuilder):
    def comment(self, data):
        self.start(ET.Comment, {})
        self.data(data)
        self.end(ET.Comment)

def patch_configmap_json(configmap_json):
    core_site = 'core-site.xml'
    cs = configmap_json['data'][core_site]

    parser = ET.XMLParser(target=CommentedTreeBuilder())
    csxml = ET.fromstring(cs, parser=parser)
    props = [prop.find('value') for prop in csxml.findall(
        "./property/name/..[name='hadoop.security.key.provider.path']")]

    if len(props) == 0:
        prop = ET.SubElement(csxml, 'property')
        name = ET.SubElement(prop, 'name')
        name.text = 'hadoop.security.key.provider.path'
        value = ET.SubElement(prop, 'value')
        key_provider_path_tag = value
    else:
        key_provider_path_tag = props[0]

    # Get HDFS Name Node FQDN repeated 5 times so that HDFS clients will
    # retry when single name Name Node is unavailable.
    #
    repeat_fqdn_n_times = 5
    name_node_fqdn = get_host_fqdn(hdfs_name_node_pod_name)
    name_node_fqdn = ';'.join([name_node_fqdn] * repeat_fqdn_n_times)

    key_provider_path_tag.text = f'kms://https@{name_node_fqdn}:9600/kms'
    cs = ET.tostring(csxml, encoding='utf-8').decode('utf-8')
    configmap_json['data'][core_site] = cs

    kms_site = 'kms-site.xml.tmpl'
    ks = configmap_json['data'][kms_site]

    kp_uri_regex = re.compile('(<name>hadoop.kms.key.provider.uri</name>\s*<value>\s*)(.*)(\s*</value>)', re.MULTILINE)

    hdfsvault_svc_fqdn = get_host_fqdn('hdfsvault-svc')

    def replace_uri(match_obj):
        if match_obj.group(2) == 'jceks://file@/var/run/secrets/keystores/kms/kms.jceks':
            return match_obj.group(1) + f'bdc://https@{hdfsvault_svc_fqdn}' + match_obj.group(3)
        return match_obj.group(0)

    ks = kp_uri_regex.sub(replace_uri, ks)

    configmap_json['data'][kms_site] = ks
    return json.dumps(configmap_json, indent=4, sort_keys=True)

def patch_configmap_if_needed(configmap_name):
    configmap = read_configmap(configmap_name)

    # Fix for model objects not valid for serialization
    #
    configmap = kubernetes.client.ApiClient().sanitize_for_serialization(configmap)
    pretty_configmap = json.dumps(configmap, indent=4, sort_keys=True)

    patched_configmap = patch_configmap_json(configmap)

    diff_lines = [line for line in difflib.context_diff(pretty_configmap.split(), patched_configmap.split(), fromfile='before patch', tofile='after patch')]
    if len(diff_lines) > 0:
        print(f'Diff between configmap {configmap_name} before and after patch:')
        for line in diff_lines:
            print(line)

        replace_configmap(configmap_name, patched_configmap)
        print(f'Applying patch for configmap {configmap_name} to enable support of HDFS encryption zones with BDC key provider.')
    else:
        print(f'Configmap {configmap_name} is already patched to enable support of HDFS encryption zones with BDC key provider.')

can_be_applied, reason = can_configmap_patch_be_applied()
if can_be_applied:
    for configmap_name in [storage_configmap_name, 'mssql-hadoop-sparkhead-configmap', 'mssql-hadoop-spark-0-configmap']:
        if configmap_exist(configmap_name):
            patch_configmap_if_needed(configmap_name)
        else:
            print(f'Skipping configmap {configmap_name} because it does not exist on this BDC deployment.')
else:
    print(reason)

### Restart pods to apply changes of configmaps

### Common functions

Define helper functions used in this notebook.

In [ ]:
# Define `run` function for transient fault handling, suggestions on error, and scrolling updates on Windows
import sys
import os
import re
import platform
import shlex
import shutil
import datetime

from subprocess import Popen, PIPE
from IPython.display import Markdown

retry_hints = {} # Output in stderr known to be transient, therefore automatically retry
error_hints = {} # Output in stderr where a known SOP/TSG exists which will be HINTed for further help
install_hint = {} # The SOP to help install the executable if it cannot be found

def run(cmd, return_output=False, no_output=False, retry_count=0, base64_decode=False, return_as_json=False, regex_mask=None):
    """Run shell command, stream stdout, print stderr and optionally return output

    NOTES:

    1.  Commands that need this kind of ' quoting on Windows e.g.:

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='data-pool')].metadata.name}

        Need to actually pass in as '"':

            kubectl get nodes -o jsonpath={.items[?(@.metadata.annotations.pv-candidate=='"'data-pool'"')].metadata.name}

        The ' quote approach, although correct when pasting into Windows cmd, will hang at the line:
        
            `iter(p.stdout.readline, b'')`

        The shlex.split call does the right thing for each platform, just use the '"' pattern for a '
    """
    MAX_RETRIES = 5
    output = ""
    retry = False

    # When running `azdata sql query` on Windows, replace any \n in """ strings, with " ", otherwise we see:
    #
    #    ('HY090', '[HY090] [Microsoft][ODBC Driver Manager] Invalid string or buffer length (0) (SQLExecDirectW)')
    #
    if platform.system() == "Windows" and cmd.startswith("azdata sql query"):
        cmd = cmd.replace("\n", " ")

    # shlex.split is required on bash and for Windows paths with spaces
    #
    cmd_actual = shlex.split(cmd)

    # Store this (i.e. kubectl, python etc.) to support binary context aware error_hints and retries
    #
    user_provided_exe_name = cmd_actual[0].lower()

    # When running python, use the python in the ADS sandbox ({sys.executable})
    #
    if cmd.startswith("python "):
        cmd_actual[0] = cmd_actual[0].replace("python", sys.executable)

        # On Mac, when ADS is not launched from terminal, LC_ALL may not be set, which causes pip installs to fail
        # with:
        #
        #    UnicodeDecodeError: 'ascii' codec can't decode byte 0xc5 in position 4969: ordinal not in range(128)
        #
        # Setting it to a default value of "en_US.UTF-8" enables pip install to complete
        #
        if platform.system() == "Darwin" and "LC_ALL" not in os.environ:
            os.environ["LC_ALL"] = "en_US.UTF-8"

    # When running `kubectl`, if AZDATA_OPENSHIFT is set, use `oc`
    #
    if cmd.startswith("kubectl ") and "AZDATA_OPENSHIFT" in os.environ:
        cmd_actual[0] = cmd_actual[0].replace("kubectl", "oc")

    # To aid supportability, determine which binary file will actually be executed on the machine
    #
    which_binary = None

    # Special case for CURL on Windows.  The version of CURL in Windows System32 does not work to
    # get JWT tokens, it returns "(56) Failure when receiving data from the peer".  If another instance
    # of CURL exists on the machine use that one.  (Unfortunately the curl.exe in System32 is almost
    # always the first curl.exe in the path, and it can't be uninstalled from System32, so here we
    # look for the 2nd installation of CURL in the path)
    if platform.system() == "Windows" and cmd.startswith("curl "):
        path = os.getenv('PATH')
        for p in path.split(os.path.pathsep):
            p = os.path.join(p, "curl.exe")
            if os.path.exists(p) and os.access(p, os.X_OK):
                if p.lower().find("system32") == -1:
                    cmd_actual[0] = p
                    which_binary = p
                    break

    # Find the path based location (shutil.which) of the executable that will be run (and display it to aid supportability), this
    # seems to be required for .msi installs of azdata.cmd/az.cmd.  (otherwise Popen returns FileNotFound) 
    #
    # NOTE: Bash needs cmd to be the list of the space separated values hence shlex.split.
    #
    if which_binary == None:
        which_binary = shutil.which(cmd_actual[0])

    # Display an install HINT, so the user can click on a SOP to install the missing binary
    #
    if which_binary == None:
        print(f"The path used to search for '{cmd_actual[0]}' was:")
        print(sys.path)

        if user_provided_exe_name in install_hint and install_hint[user_provided_exe_name] is not None:
            display(Markdown(f'HINT: Use [{install_hint[user_provided_exe_name][0]}]({install_hint[user_provided_exe_name][1]}) to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)")
    else:   
        cmd_actual[0] = which_binary

    start_time = datetime.datetime.now().replace(microsecond=0)

    cmd_display = cmd
    if regex_mask is not None:
        regex = re.compile(regex_mask)
        cmd_display = re.sub(regex, '******', cmd)
        
    print(f"START: {cmd_display} @ {start_time} ({datetime.datetime.utcnow().replace(microsecond=0)} UTC)")
    print(f"       using: {which_binary} ({platform.system()} {platform.release()} on {platform.machine()})")
    print(f"       cwd: {os.getcwd()}")

    # Command-line tools such as CURL and AZDATA HDFS commands output
    # scrolling progress bars, which causes Jupyter to hang forever, to
    # workaround this, use no_output=True
    #

    # Work around a infinite hang when a notebook generates a non-zero return code, break out, and do not wait
    #
    wait = True 

    try:
        if no_output:
            p = Popen(cmd_actual)
        else:
            p = Popen(cmd_actual, stdout=PIPE, stderr=PIPE, bufsize=1)
            with p.stdout:
                for line in iter(p.stdout.readline, b''):
                    line = line.decode()
                    if return_output:
                        output = output + line
                    else:
                        if cmd.startswith("azdata notebook run"): # Hyperlink the .ipynb file
                            regex = re.compile('  "(.*)"\: "(.*)"') 
                            match = regex.match(line)
                            if match:
                                if match.group(1).find("HTML") != -1:
                                    display(Markdown(f' - "{match.group(1)}": "{match.group(2)}"'))
                                else:
                                    display(Markdown(f' - "{match.group(1)}": "[{match.group(2)}]({match.group(2)})"'))

                                    wait = False
                                    break # otherwise infinite hang, have not worked out why yet.
                        else:
                            print(line, end='')

        if wait:
            p.wait()
    except FileNotFoundError as e:
        if install_hint is not None:
            display(Markdown(f'HINT: Use {install_hint} to resolve this issue.'))

        raise FileNotFoundError(f"Executable '{cmd_actual[0]}' not found in path (where/which)") from e

    exit_code_workaround = 0 # WORKAROUND: azdata hangs on exception from notebook on p.wait()

    if not no_output:
        for line in iter(p.stderr.readline, b''):
            try:
                line_decoded = line.decode()
            except UnicodeDecodeError:
                # NOTE: Sometimes we get characters back that cannot be decoded(), e.g.
                #
                #   \xa0
                #
                # For example see this in the response from `az group create`:
                #
                # ERROR: Get Token request returned http error: 400 and server 
                # response: {"error":"invalid_grant",# "error_description":"AADSTS700082: 
                # The refresh token has expired due to inactivity.\xa0The token was 
                # issued on 2018-10-25T23:35:11.9832872Z
                #
                # which generates the exception:
                #
                # UnicodeDecodeError: 'utf-8' codec can't decode byte 0xa0 in position 179: invalid start byte
                #
                print("WARNING: Unable to decode stderr line, printing raw bytes:")
                print(line)
                line_decoded = ""
                pass
            else:

                # azdata emits a single empty line to stderr when doing an hdfs cp, don't
                # print this empty "ERR:" as it confuses.
                #
                if line_decoded == "":
                    continue
                
                print(f"STDERR: {line_decoded}", end='')

                if line_decoded.startswith("An exception has occurred") or line_decoded.startswith("ERROR: An error occurred while executing the following cell"):
                    exit_code_workaround = 1

                # inject HINTs to next TSG/SOP based on output in stderr
                #
                if user_provided_exe_name in error_hints:
                    for error_hint in error_hints[user_provided_exe_name]:
                        if line_decoded.find(error_hint[0]) != -1:
                            display(Markdown(f'HINT: Use [{error_hint[1]}]({error_hint[2]}) to resolve this issue.'))

                # Verify if a transient error, if so automatically retry (recursive)
                #
                if user_provided_exe_name in retry_hints:
                    for retry_hint in retry_hints[user_provided_exe_name]:
                        if line_decoded.find(retry_hint) != -1:
                            if retry_count < MAX_RETRIES:
                                print(f"RETRY: {retry_count} (due to: {retry_hint})")
                                retry_count = retry_count + 1
                                output = run(cmd, return_output=return_output, retry_count=retry_count)

                                if return_output:
                                    if base64_decode:
                                        import base64
                                        return base64.b64decode(output).decode('utf-8')
                                    else:
                                        return output

    elapsed = datetime.datetime.now().replace(microsecond=0) - start_time

    # WORKAROUND: We avoid infinite hang above in the `azdata notebook run` failure case, by inferring success (from stdout output), so
    # don't wait here, if success known above
    #
    if wait: 
        if p.returncode != 0:
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(p.returncode)}.\n')
    else:
        if exit_code_workaround !=0 :
            raise SystemExit(f'Shell command:\n\n\t{cmd_display} ({elapsed}s elapsed)\n\nreturned non-zero exit code: {str(exit_code_workaround)}.\n')

    print(f'\nSUCCESS: {elapsed}s elapsed.\n')

    if return_output:
        if base64_decode:
            import base64
            return base64.b64decode(output).decode('utf-8')
        else:
            return output



# Hints for tool retry (on transient fault), known errors and install guide
#
retry_hints = {'azdata': ['Endpoint sql-server-master does not exist', 'Endpoint livy does not exist', 'Failed to get state for cluster', 'Endpoint webhdfs does not exist', 'Adaptive Server is unavailable or does not exist', 'Error: Address already in use', 'Login timeout expired (0) (SQLDriverConnect)', 'SSPI Provider: No Kerberos credentials available',  ], 'kubectl': ['A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond',  ], 'python': [ ], }
error_hints = {'azdata': [['Please run \'azdata login\' to first authenticate', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['The token is expired', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Reason: Unauthorized', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Max retries exceeded with url: /api/v1/bdc/endpoints', 'SOP028 - azdata login', '../common/sop028-azdata-login.ipynb'], ['Look at the controller logs for more details', 'TSG027 - Observe cluster deployment', '../diagnose/tsg027-observe-bdc-create.ipynb'], ['provided port is already allocated', 'TSG062 - Get tail of all previous container logs for pods in BDC namespace', '../log-files/tsg062-tail-bdc-previous-container-logs.ipynb'], ['Create cluster failed since the existing namespace', 'SOP061 - Delete a big data cluster', '../install/sop061-delete-bdc.ipynb'], ['Failed to complete kube config setup', 'TSG067 - Failed to complete kube config setup', '../repair/tsg067-failed-to-complete-kube-config-setup.ipynb'], ['Data source name not found and no default driver specified', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Can\'t open lib \'ODBC Driver 17 for SQL Server', 'SOP069 - Install ODBC for SQL Server', '../install/sop069-install-odbc-driver-for-sql-server.ipynb'], ['Control plane upgrade failed. Failed to upgrade controller.', 'TSG108 - View the controller upgrade config map', '../diagnose/tsg108-controller-failed-to-upgrade.ipynb'], ['NameError: name \'azdata_login_secret_name\' is not defined', 'SOP013 - Create secret for azdata login (inside cluster)', '../common/sop013-create-secret-for-azdata-login.ipynb'], ['ERROR: No credentials were supplied, or the credentials were unavailable or inaccessible.', 'TSG124 - \'No credentials were supplied\' error from azdata login', '../repair/tsg124-no-credentials-were-supplied.ipynb'], ['Please accept the license terms to use this product through', 'TSG126 - azdata fails with \'accept the license terms to use this product\'', '../repair/tsg126-accept-license-terms.ipynb'],  ], 'kubectl': [['no such host', 'TSG010 - Get configuration contexts', '../monitor-k8s/tsg010-get-kubernetes-contexts.ipynb'], ['No connection could be made because the target machine actively refused it', 'TSG056 - Kubectl fails with No connection could be made because the target machine actively refused it', '../repair/tsg056-kubectl-no-connection-could-be-made.ipynb'],  ], 'python': [['Library not loaded: /usr/local/opt/unixodbc', 'SOP012 - Install unixodbc for Mac', '../install/sop012-brew-install-odbc-for-sql-server.ipynb'], ['WARNING: You are using pip version', 'SOP040 - Upgrade pip in ADS Python sandbox', '../install/sop040-upgrade-pip.ipynb'],  ], }
install_hint = {'azdata': [ 'SOP063 - Install azdata CLI (using package manager)', '../install/sop063-packman-install-azdata.ipynb' ],  'kubectl': [ 'SOP036 - Install kubectl command line interface', '../install/sop036-install-kubectl.ipynb' ],  }


print('Common functions defined successfully.')

### Parameters for waiting for healthy cluster state

In [ ]:
timeout = 600  # amount of time in seconds to wait before cluster is healthy:  default to 10 minutes
check_interval = 30  # amount of time in seconds between health checks - default 30 seconds
min_pod_count = 10  # minimum number of healthy pods required to assert health

### Helper functions for waiting for the cluster to become healthy

In [ ]:
import threading
import time
import sys
import os
from IPython.display import Markdown

isRunning = True

def all_containers_ready(pod):
    """helper method returns true if all the containers within the given pod are ready

    Arguments:
        pod {v1Pod} -- Metadata retrieved from the api call to.
    """
         
    return all(map(lambda c: c.ready is True, pod.status.container_statuses))


def pod_is_ready(pod):
    """tests that the pod, and all containers are ready

    Arguments:
        pod {v1Pod} -- Metadata retrieved from api call.
    """

    return "job-name" in pod.metadata.labels or (pod.status.phase == "Running" and all_containers_ready(pod))


def waitReady():
    """Waits for all pods, and containers to become ready.
    """
    while isRunning:
        try:
            time.sleep(check_interval)
            pods = get_pods()
            allReady = len(pods.items) >= min_pod_count and all(map(pod_is_ready, pods.items))

            if allReady:
                return True
            else:
                display(Markdown(get_pod_failures(pods)))
                display(Markdown(f"cluster not healthy, rechecking in {check_interval} seconds."))
        except Exception as ex:
            last_error_message = str(ex)
            display(Markdown(last_error_message))
            time.sleep(check_interval)

def get_pod_failures(pods=None):
    """Returns a status message for any pods that are not ready.
    """
    results = ""
    if not pods:
        pods = get_pods()

    for pod in pods.items:
        if "job-name" not in pod.metadata.labels:
            if pod.status and pod.status.container_statuses:
                for container in filter(lambda c: c.ready is False, pod.status.container_statuses):
                    results = results + "Container {0} in Pod {1} is not ready. Reported status: {2} <br/>".format(container.name, pod.metadata.name, container.state)       
            else:
                results = results + "Pod {0} is not ready.  <br/>".format(pod.metadata.name)
    return results


def get_pods():
    """Returns a list of pods by namespace, or all namespaces if no namespace is specified
    """
    pods = None
    if namespace is not None:
        display(Markdown(f'Checking namespace {namespace}'))
        pods = api.list_namespaced_pod(namespace, _request_timeout=30) 
    else:
        display(Markdown('Checking all namespaces'))
        pods = api.list_pod_for_all_namespaces(_request_timeout=30)
    return pods

def wait_for_cluster_healthy():
    isRunning = True
    mt = threading.Thread(target=waitReady)
    mt.start()
    mt.join(timeout=timeout)

    if mt.is_alive():
      raise SystemExit("Timeout waiting for all cluster to be healthy.")
      
    isRunning = False

In [ ]:
import re

def restart_pods_if_needed():
    # Prevent unnecessary pod restart on deployments where HDFS encryption zones cannot be enabled.
    #
    can_be_applied, reason = can_configmap_patch_be_applied()
    if not can_be_applied:
        print(reason)
        return

    pods_list_string = run(f'kubectl get pod -n {namespace} -o=name', return_output=True)

    # Remember count of healthy pods
    #
    min_pod_count = len(get_pods().items)

    pod_regex = re.compile('pod/(?P<pod>(nmnode-0|storage-0|sparkhead|spark-0)-[0-9]+)\s*')
    pods = (pod_match.group('pod') for pod_match in pod_regex.finditer(pods_list_string))
    for pod in pods:
        print(f'Restarting pod {pod}')
        run(f'kubectl delete pod {pod} -n {namespace}')

        # Do not restart more than one pod at once
        #
        wait_for_cluster_healthy()

    print("Restarted pods. HDFS Encryption zones can now be used in the cluster.")
    
restart_pods_if_needed()

In [ ]:
print("Notebook execution is complete.")